# Applications of Vectorization

# Spam Filter - Count Vectorization Method

In [1]:
import pandas as pd
import numpy as np

## Import the Data

Import the data and take a look at it.

In [2]:
url = "https://raw.githubusercontent.com/ryanleeallred/datasets/master/spam.csv"

df = pd.read_csv(url, encoding="ISO-8859-1")
print(df.shape)
df.head()

(5572, 5)


,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


## Tidy up initial DataFrame

- Change Pandas display options so that we can see more of the text
- Drop the unnamed columns, I'm not sure why they're in there, but we don't need them.
- Rename the v1 and v2 columns.

In [3]:
pd.set_option('display.max_colwidth', 200)
df = df.drop(["Unnamed: 2", "Unnamed: 3", "Unnamed: 4"], axis=1)
df = df.rename(columns={"v1":"label", "v2":"text"})
df.head()

,label,text
0,ham,"Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives around here though"


You'll notice right of the bat that this text isn't as coherent as the job listings. We'll proceed like normal though. 

What is the ratio of Spam to Ham messages?

In [4]:
df.label.value_counts()

ham     4825
spam     747
Name: label, dtype: int64

## Categorical encoding on labels.

In [5]:
df['label_num'] = df.label.map({'ham': 0, 'spam': 1})

df.head()

,label,text,label_num
0,ham,"Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...",0
1,ham,Ok lar... Joking wif u oni...,0
2,spam,Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's,1
3,ham,U dun say so early hor... U c already then say...,0
4,ham,"Nah I don't think he goes to usf, he lives around here though",0


## Model Validation - Train Test Split (quick and dirty)
Since we're going to do some modeling we're going to need some model validation. For simplicity lets just do a quick train_test_split for today. You can try out Cross Validation on your assignment today, I just want to get to a quick baseline. 

In [6]:
from sklearn.model_selection import train_test_split

X = df.text
y = df.label_num

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

Look at sizes of our train and test datasets

In [7]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(4457,)
(1115,)
(4457,)
(1115,)


## Count Vectorizer

Today we're just going to let Scikit-Learn do our text cleaning and preprocessing for us.

Lets run our vectorizer on our text messages and take a peek at the tokenization of the vocabulary

In [8]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(max_features=None, ngram_range=(1,1), stop_words='english')

vectorizer.fit(X_train)

print(vectorizer.vocabulary_)

{'boat': 1371, 'moms': 4416, 'check': 1706, 'yo': 7415, 'half': 3210, 'naked': 4520, 'bank': 1187, 'granite': 3140, 'issues': 3631, 'strong': 6296, 'buy': 1533, 'explosive': 2661, 'pick': 5005, 'members': 4292, '300': 419, 'nasdaq': 4533, 'symbol': 6440, 'cdgt': 1649, '00': 0, 'giving': 3061, 'second': 5770, 'chance': 1674, 'rahul': 5351, 'dengra': 2190, 'played': 5048, 'smash': 6027, 'bros': 1476, 'lt': 4109, 'gt': 3174, 'religiously': 5484, 'private': 5225, '2003': 345, 'account': 781, 'statement': 6232, '07973788240': 45, 'shows': 5921, '800': 630, 'redeemed': 5453, 'points': 5096, '08715203649': 122, 'identifier': 3483, 'code': 1831, '40533': 476, 'expires': 2656, '31': 430, '10': 255, '04': 15, 'says': 5728, 'answer': 954, 'texts': 6566, 'confirm': 1912, 'deny': 2196, '88066': 695, 'lost': 4073, '3pound': 463, 'help': 3301, 'okey': 4742, 'dokey': 2346, 'll': 4022, 'bit': 1323, 'just': 3752, 'sorting': 6106, 'stuff': 6311, 'come': 1858, 'people': 4953, 'wah': 7083, 'lucky': 4115, '

Now we'll complete the vectorization by running .transform() and then save the results to a dataframe for viewing.
You don't need to save it to a dataframe, you can use most ML models with just the 2D array output.

That's a lot of columns.

In [9]:
train_word_counts = vectorizer.transform(X_train)

X_train_vectorized = pd.DataFrame(train_word_counts.toarray(), columns=vectorizer.get_feature_names())

print(X_train_vectorized.shape)
X_train_vectorized.head()

(4457, 7472)


,00,000,000pes,008704050406,0089,0121,01223585236,0125698789,02,0207,...,ìï,û_,û_thanks,ûªm,ûªt,ûªve,ûï,ûïharry,ûò,ûówell
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


We also need to vectorize our X_test data, but we need to use the same vocabulary as the training dataset, so we'll just call .transform() on X_test to get our vectorized X_test_df

In [10]:
test_word_counts = vectorizer.transform(X_test)

X_test_vectorized = pd.DataFrame(test_word_counts.toarray(), columns=vectorizer.get_feature_names())

print(X_test_vectorized.shape)
X_test_vectorized.head()

(1115, 7472)


,00,000,000pes,008704050406,0089,0121,01223585236,0125698789,02,0207,...,ìï,û_,û_thanks,ûªm,ûªt,ûªve,ûï,ûïharry,ûò,ûówell
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Lets run some classification models and see what kind of accuracy we can get!

## Logistic Regression

In [11]:
from sklearn.linear_model import LogisticRegression

LR = LogisticRegression(random_state=42).fit(X_train_vectorized, y_train)

train_predictions = LR.predict(X_train_vectorized)
test_predictions = LR.predict(X_test_vectorized)

Now we'll evaluate both our training and testing accuracy. 

In [12]:
from sklearn.metrics import accuracy_score

print(f'Train Accuracy: {accuracy_score(y_train, train_predictions)}')
print(f'Test Accuracy: {accuracy_score(y_test, test_predictions)}')

Train Accuracy: 0.9964101413506843
Test Accuracy: 0.9775784753363229


## Multinomial Naive Bayes

In [13]:
from sklearn.naive_bayes import MultinomialNB

MNB = MultinomialNB().fit(X_train_vectorized, y_train)

train_predictions = MNB.predict(X_train_vectorized)
test_predictions = MNB.predict(X_test_vectorized)

print(f'Train Accuracy: {accuracy_score(y_train, train_predictions)}')
print(f'Test Accuracy: {accuracy_score(y_test, test_predictions)}')

Train Accuracy: 0.9946152120260264
Test Accuracy: 0.9838565022421525


## Random Forest Classifier

In [14]:
from sklearn.ensemble import RandomForestClassifier

RFC = RandomForestClassifier().fit(X_train_vectorized, y_train)

train_predictions = RFC.predict(X_train_vectorized)
test_predictions = RFC.predict(X_test_vectorized)

print(f'Train Accuracy: {accuracy_score(y_train, train_predictions)}')
print(f'Test Accuracy: {accuracy_score(y_test, test_predictions)}')

/Users/ryanallred/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


Train Accuracy: 0.9966345075162666
Test Accuracy: 0.968609865470852


# Spam Filter - TF-IDF Vectorization Method

In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(max_features=None, ngram_range=(1,1), stop_words='english')

vectorizer.fit(X_train)

print(vectorizer.vocabulary_)

{'boat': 1371, 'moms': 4416, 'check': 1706, 'yo': 7415, 'half': 3210, 'naked': 4520, 'bank': 1187, 'granite': 3140, 'issues': 3631, 'strong': 6296, 'buy': 1533, 'explosive': 2661, 'pick': 5005, 'members': 4292, '300': 419, 'nasdaq': 4533, 'symbol': 6440, 'cdgt': 1649, '00': 0, 'giving': 3061, 'second': 5770, 'chance': 1674, 'rahul': 5351, 'dengra': 2190, 'played': 5048, 'smash': 6027, 'bros': 1476, 'lt': 4109, 'gt': 3174, 'religiously': 5484, 'private': 5225, '2003': 345, 'account': 781, 'statement': 6232, '07973788240': 45, 'shows': 5921, '800': 630, 'redeemed': 5453, 'points': 5096, '08715203649': 122, 'identifier': 3483, 'code': 1831, '40533': 476, 'expires': 2656, '31': 430, '10': 255, '04': 15, 'says': 5728, 'answer': 954, 'texts': 6566, 'confirm': 1912, 'deny': 2196, '88066': 695, 'lost': 4073, '3pound': 463, 'help': 3301, 'okey': 4742, 'dokey': 2346, 'll': 4022, 'bit': 1323, 'just': 3752, 'sorting': 6106, 'stuff': 6311, 'come': 1858, 'people': 4953, 'wah': 7083, 'lucky': 4115, '

## Vectorize training data

In [16]:
train_word_counts = vectorizer.transform(X_train)

X_train_vectorized = pd.DataFrame(train_word_counts.toarray(), columns=vectorizer.get_feature_names())

print(X_train_vectorized.shape)
X_train_vectorized.head()

(4457, 7472)


,00,000,000pes,008704050406,0089,0121,01223585236,0125698789,02,0207,...,ìï,û_,û_thanks,ûªm,ûªt,ûªve,ûï,ûïharry,ûò,ûówell
0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.265494,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Vectorize testing data

In [17]:
test_word_counts = vectorizer.transform(X_test)

X_test_vectorized = pd.DataFrame(test_word_counts.toarray(), columns=vectorizer.get_feature_names())

print(X_test_vectorized.shape)
X_test_vectorized.head()

(1115, 7472)


,00,000,000pes,008704050406,0089,0121,01223585236,0125698789,02,0207,...,ìï,û_,û_thanks,ûªm,ûªt,ûªve,ûï,ûïharry,ûò,ûówell
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Logistic Regression

In [18]:
LR = LogisticRegression(random_state=42).fit(X_train_vectorized, y_train)

train_predictions = LR.predict(X_train_vectorized)
test_predictions = LR.predict(X_test_vectorized)

print(f'Train Accuracy: {accuracy_score(y_train, train_predictions)}')
print(f'Test Accuracy: {accuracy_score(y_test, test_predictions)}')

Train Accuracy: 0.9694862014808167
Test Accuracy: 0.9524663677130045


## Multinomial Naive Bayes

In [19]:
from sklearn.naive_bayes import MultinomialNB

MNB = MultinomialNB().fit(X_train_vectorized, y_train)

train_predictions = MNB.predict(X_train_vectorized)
test_predictions = MNB.predict(X_test_vectorized)

print(f'Train Accuracy: {accuracy_score(y_train, train_predictions)}')
print(f'Test Accuracy: {accuracy_score(y_test, test_predictions)}')

Train Accuracy: 0.9838456360780794
Test Accuracy: 0.9668161434977578


## Random Forest Classifier

In [20]:
from sklearn.ensemble import RandomForestClassifier

RFC = RandomForestClassifier().fit(X_train_vectorized, y_train)

train_predictions = RFC.predict(X_train_vectorized)
test_predictions = RFC.predict(X_test_vectorized)

print(f'Train Accuracy: {accuracy_score(y_train, train_predictions)}')
print(f'Test Accuracy: {accuracy_score(y_test, test_predictions)}')

Train Accuracy: 0.99798070450976
Test Accuracy: 0.9704035874439462


# Sentiment Analysis

## What is Sentiment Analysis?

The objective of sentiment analysis is to take a phrase and based on the text of the phrase determine if its sentiment is: Postive, Neutral, or Negative. 

Suppose that you wanted to use NLP to classify reviews for your company's products as either positive, neutral, or negative. Maybe you don't trust the star ratings left by the users and you want an additional measure of sentiment from each review - maybe you would use this as a feature generation technique for additional modeling, or to identify disgruntled customers and reach out to them to improve your customer service, etc. Sentiment Analysis has also been used heavily in stock market price estimation by trying to track the sentiment of the tweets of individuals after breaking news comes out about a company.

Does every word in each review contribute to its overall sentiment? Not really. Stop words for example don't really tell us much about the overall sentiment of the text, so just like we did before, we will discard them. 

## NLTK Movie Review Sentiment Analysis

In [21]:
!pip install -U nltk

import nltk
nltk.download('movie_reviews')
nltk.download('stopwords')
from nltk.corpus import movie_reviews
import random

Requirement already up-to-date: nltk in /Users/ryanallred/anaconda3/lib/python3.6/site-packages (3.4)


[nltk_data] Downloading package movie_reviews to
[nltk_data]     /Users/ryanallred/nltk_data...
[nltk_data]   Package movie_reviews is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/ryanallred/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Check that we have movie reviews

In [22]:
# How many total reviews are there?
print("Total reviews:", len(movie_reviews.fileids()))

# Total positive reviews
print("Positive reviews:", len(movie_reviews.fileids('pos'))) 
 
# Total negative reviews
print("Negative reviews:", len(movie_reviews.fileids('neg')))

Total reviews: 2000
Positive reviews: 1000
Negative reviews: 1000


## Get Reviews and randomize

In [23]:
reviews = [(list(movie_reviews.words(fileid)), category)
              for category in movie_reviews.categories()
              for fileid in movie_reviews.fileids(category)]

random.shuffle(reviews)

## Understand the format of the data

In [24]:
# Print Review Text:
print(reviews[0][0])

# Print Review Sentiment:
print(reviews[0][1])

# Print Review Text:
print(reviews[1][0])

# Print Review Sentiment:
print(reviews[1][1])

['"', 'in', 'dreams', '"', 'might', 'keep', 'you', 'awake', 'at', 'night', ',', 'but', 'not', 'because', 'of', 'its', 'creepy', 'imagery', ',', 'bizarre', 'visual', 'style', 'or', 'story', 'about', 'a', 'clairvoyant', 'madman', 'who', 'lures', 'young', 'girls', 'to', 'their', 'untimely', 'deaths', '.', 'no', ',', 'the', 'source', 'of', 'potential', 'sleeplessness', 'here', 'lies', 'within', 'the', 'movie', "'", 's', 'brutally', 'squandered', 'potential', ',', 'the', 'least', 'of', 'which', 'is', 'an', 'admittedly', 'nifty', 'premise', '-', 'even', 'by', 'tired', 'serial', 'killer', 'genre', 'standards', '.', 'the', 'big', 'letdown', ',', 'however', ',', 'comes', 'upon', 'the', 'realization', 'that', 'this', '100', '-', 'minute', 'head', '-', 'scratcher', 'was', 'masterminded', 'by', 'neil', 'jordan', ',', 'the', 'man', 'behind', '"', 'the', 'crying', 'game', '.', '"', 'he', "'", 's', 'no', 'stranger', 'to', 'cinematic', 'weirdness', ',', 'but', 'this', 'nutty', 'nonsense', 'really', 'p

## Add reviews to a dataframe for kicks

In [25]:
documents = []
sentiments = []

for review in reviews:
  
  # Add sentiment to list
  if review[1] == "pos":
    sentiments.append(1)
  else:
    sentiments.append(0)
  
  # Add text to list
  review_text = " ".join(review[0])
  documents.append(review_text)
  
df = pd.DataFrame({"text": documents, "sentiment": sentiments})
df.head()

,sentiment,text
0,0,""" in dreams "" might keep you awake at night , but not because of its creepy imagery , bizarre visual style or story about a clairvoyant madman who lures young girls to their untimely deaths . no ,..."
1,1,"this three hour movie opens up with a view of singer / guitar player / musician / composer frank zappa rehearsing with his fellow band members . all the rest displays a compilation of footage , mo..."
2,1,"gere , willis , poitier chase each other around the world the jackal a film review by michael redman copyright 1997 by michael redman when the soviet union imploded , the western countries lost th..."
3,1,"written by john grisham and robert altman ( as al hayes ) directed by robert altman some time ago , a young director was given the opportunity to direct the screen version of a popular novel . the..."
4,1,capsule : a short punchy action sequel to the two dinosaur films made by steven spielberg . joe johnston directs a straightforward story of an excursion back to the island of the dinosaurs . it le...


## Train Test Split

In [26]:
X = df.text
y = df.sentiment

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Sentiment Analysis - CountVectorizer

## Generate vocabulary from train dataset

In [27]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(max_features=None, ngram_range=(1,1), stop_words='english')

vectorizer.fit(X_train)

print(vectorizer.vocabulary_)

{'writer': 35862, 'director': 9013, 'lawrence': 18370, 'kasdan': 17629, 'hand': 14441, 'penning': 23538, 'biggest': 3457, 'film': 12062, 'successes': 31218, '1980s': 177, 'yes': 36008, 'right': 27106, 'wrote': 35881, 'raiders': 25679, 'lost': 19110, 'ark': 2019, '10': 8, 'empire': 10549, 'strikes': 30975, 'return': 26888, 'jedi': 17220, 'looks': 19057, 'decided': 8183, 'test': 32258, 'skills': 29373, 'mediocre': 20167, 'screenwriting': 28242, 'bland': 3618, 'directing': 9003, 'guess': 14189, 'succeeds': 31216, 'plot': 24184, 'man': 19559, 'shady': 28694, 'regrettable': 26291, 'past': 23294, 'decides': 8186, 'run': 27575, 'away': 2558, 'small': 29615, 'american': 1504, 'town': 32906, 'pretends': 24807, 'licensed': 18691, 'psychologist': 25286, 'open': 22462, 'armed': 2034, 'swarm': 31596, 'people': 23555, 'problems': 24936, 'isn': 17042, 'long': 19029, 'befriends': 3164, 'billionaire': 3487, 'folks': 12491, 'suspicious': 31549, 'falls': 11647, 'patients': 23337, 'critique': 7587, 'comed

## Generate Vectorizations

In [28]:
train_word_counts = vectorizer.transform(X_train)

X_train_vectorized = pd.DataFrame(train_word_counts.toarray(), columns=vectorizer.get_feature_names())

print(X_train_vectorized.shape)
X_train_vectorized.head()

(1600, 36212)


,00,000,0009f,007,03,04,05,05425,10,100,...,zuehlke,zuko,zukovsky,zundel,zurg,zus,zweibel,zwick,zwigoff,zycie
0,0,0,0,0,0,0,0,0,8,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [29]:
test_word_counts = vectorizer.transform(X_test)

X_test_vectorized = pd.DataFrame(test_word_counts.toarray(), columns=vectorizer.get_feature_names())

print(X_test_vectorized.shape)
X_test_vectorized.head()

(400, 36212)


,00,000,0009f,007,03,04,05,05425,10,100,...,zuehlke,zuko,zukovsky,zundel,zurg,zus,zweibel,zwick,zwigoff,zycie
0,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Logistic Regression

In [30]:
LR = LogisticRegression(random_state=42).fit(X_train_vectorized, y_train)

train_predictions = LR.predict(X_train_vectorized)
test_predictions = LR.predict(X_test_vectorized)

print(f'Train Accuracy: {accuracy_score(y_train, train_predictions)}')
print(f'Test Accuracy: {accuracy_score(y_test, test_predictions)}')

Train Accuracy: 1.0
Test Accuracy: 0.8525


## Multinomial Naive Bayes

In [31]:
MNB = MultinomialNB().fit(X_train_vectorized, y_train)

train_predictions = MNB.predict(X_train_vectorized)
test_predictions = MNB.predict(X_test_vectorized)

print(f'Train Accuracy: {accuracy_score(y_train, train_predictions)}')
print(f'Test Accuracy: {accuracy_score(y_test, test_predictions)}')

Train Accuracy: 0.98375
Test Accuracy: 0.8


## Random Forest Classifier

In [32]:
RFC = RandomForestClassifier().fit(X_train_vectorized, y_train)

train_predictions = RFC.predict(X_train_vectorized)
test_predictions = RFC.predict(X_test_vectorized)

print(f'Train Accuracy: {accuracy_score(y_train, train_predictions)}')
print(f'Test Accuracy: {accuracy_score(y_test, test_predictions)}')

Train Accuracy: 0.9925
Test Accuracy: 0.705


# Sentiment Analysis - tfidfVectorizer

## Vocabulary

In [33]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(max_features=None, ngram_range=(1,1), stop_words='english')

vectorizer.fit(X_train)

print(vectorizer.vocabulary_)

{'writer': 35862, 'director': 9013, 'lawrence': 18370, 'kasdan': 17629, 'hand': 14441, 'penning': 23538, 'biggest': 3457, 'film': 12062, 'successes': 31218, '1980s': 177, 'yes': 36008, 'right': 27106, 'wrote': 35881, 'raiders': 25679, 'lost': 19110, 'ark': 2019, '10': 8, 'empire': 10549, 'strikes': 30975, 'return': 26888, 'jedi': 17220, 'looks': 19057, 'decided': 8183, 'test': 32258, 'skills': 29373, 'mediocre': 20167, 'screenwriting': 28242, 'bland': 3618, 'directing': 9003, 'guess': 14189, 'succeeds': 31216, 'plot': 24184, 'man': 19559, 'shady': 28694, 'regrettable': 26291, 'past': 23294, 'decides': 8186, 'run': 27575, 'away': 2558, 'small': 29615, 'american': 1504, 'town': 32906, 'pretends': 24807, 'licensed': 18691, 'psychologist': 25286, 'open': 22462, 'armed': 2034, 'swarm': 31596, 'people': 23555, 'problems': 24936, 'isn': 17042, 'long': 19029, 'befriends': 3164, 'billionaire': 3487, 'folks': 12491, 'suspicious': 31549, 'falls': 11647, 'patients': 23337, 'critique': 7587, 'comed

## Train

In [34]:
train_word_counts = vectorizer.transform(X_train)

X_train_vectorized = pd.DataFrame(train_word_counts.toarray(), columns=vectorizer.get_feature_names())

print(X_train_vectorized.shape)
X_train_vectorized.head()

(1600, 36212)


,00,000,0009f,007,03,04,05,05425,10,100,...,zuehlke,zuko,zukovsky,zundel,zurg,zus,zweibel,zwick,zwigoff,zycie
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.242797,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Test

In [35]:
test_word_counts = vectorizer.transform(X_test)

X_test_vectorized = pd.DataFrame(test_word_counts.toarray(), columns=vectorizer.get_feature_names())

print(X_test_vectorized.shape)
X_test_vectorized.head()

(400, 36212)


,00,000,0009f,007,03,04,05,05425,10,100,...,zuehlke,zuko,zukovsky,zundel,zurg,zus,zweibel,zwick,zwigoff,zycie
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.056877,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Logistic Regression

In [36]:
LR = LogisticRegression(random_state=42).fit(X_train_vectorized, y_train)

train_predictions = LR.predict(X_train_vectorized)
test_predictions = LR.predict(X_test_vectorized)

print(f'Train Accuracy: {accuracy_score(y_train, train_predictions)}')
print(f'Test Accuracy: {accuracy_score(y_test, test_predictions)}')

Train Accuracy: 0.983125
Test Accuracy: 0.8275


## Multinomial Naive Bayes

In [37]:
MNB = MultinomialNB().fit(X_train_vectorized, y_train)

train_predictions = MNB.predict(X_train_vectorized)
test_predictions = MNB.predict(X_test_vectorized)

print(f'Train Accuracy: {accuracy_score(y_train, train_predictions)}')
print(f'Test Accuracy: {accuracy_score(y_test, test_predictions)}')

Train Accuracy: 0.976875
Test Accuracy: 0.815


## Random Forest Classifier

In [38]:
RFC = RandomForestClassifier().fit(X_train_vectorized, y_train)

train_predictions = RFC.predict(X_train_vectorized)
test_predictions = RFC.predict(X_test_vectorized)

print(f'Train Accuracy: {accuracy_score(y_train, train_predictions)}')
print(f'Test Accuracy: {accuracy_score(y_test, test_predictions)}')

Train Accuracy: 0.99125
Test Accuracy: 0.68


# Using NLTK to clean the data

## Importing the data fresh to avoid variable collisions

In [39]:
reviews = [(list(movie_reviews.words(fileid)), category)
              for category in movie_reviews.categories()
              for fileid in movie_reviews.fileids(category)]

random.shuffle(reviews, )

In [40]:
documents = []
sentiments = []

for review in reviews:
  
  # Add sentiment to list
  if review[1] == "pos":
    sentiments.append(1)
  else:
    sentiments.append(0)
  
  # Add text to list
  review_text = " ".join(review[0])
  documents.append(review_text)
  
df = pd.DataFrame({"text": documents, "sentiment": sentiments})
df.head()

,sentiment,text
0,0,"i saw this film on christmas day expecting an upbeat comedy . boy was i in for a christmas dissapointment ! after an hour of the movie , i was ready to change rooms - into another theater ! read o..."
1,0,first and foremost i think this movie comes to grief at the hands of the pg rating it ' s been given . for those of us who haven ' t read the james lee burke novel that heaven ' s prisoners is bas...
2,1,"synopsis : retiring detective jerry black ( nicholson ) becomes involved in the case of a murdered girl , and promises her parents that he will find the killer . he purchases a gas station on a ro..."
3,0,"john boorman ' s "" zardoz "" is a goofy cinematic debacle so fundamentally misconceived and laughably executed that it takes on a bizarre enjoyment quality all its own . not since the rampant bumbl..."
4,1,"pulp fiction , quentin tarantino ' s anxiously awaited and superb follow - up to reservoir dogs , is absolutely and without a doubt progressing as one of the most talked about , loved , and hated ..."


## Cleaning function to apply to each document

In [41]:
from nltk.corpus import stopwords
import string

# turn a doc into clean tokens
def clean_doc(doc):
	# split into tokens by white space
	tokens = doc.split()
	# remove punctuation from each token
	table = str.maketrans('', '', string.punctuation)
	tokens = [w.translate(table) for w in tokens]
	# remove remaining tokens that are not alphabetic
	tokens = [word for word in tokens if word.isalpha()]
	# filter out stop words
	stop_words = set(stopwords.words('english'))
	tokens = [w for w in tokens if not w in stop_words]
	# filter out short tokens
	tokens = [word for word in tokens if len(word) > 1]
	return tokens

df_nltk = pd.DataFrame()
df_nltk['text'] = df.text.apply(clean_doc)
df_nltk['sentiment'] = df.sentiment
df_nltk.head()

,text,sentiment
0,"[saw, film, christmas, day, expecting, upbeat, comedy, boy, christmas, dissapointment, hour, movie, ready, change, rooms, another, theater, read, see, say, four, rooms, starring, tim, roth, jennif...",0
1,"[first, foremost, think, movie, comes, grief, hands, pg, rating, given, us, read, james, lee, burke, novel, heaven, prisoners, based, rabidly, go, buy, tickets, movies, featuring, combined, talent...",0
2,"[synopsis, retiring, detective, jerry, black, nicholson, becomes, involved, case, murdered, girl, promises, parents, find, killer, purchases, gas, station, route, believes, killer, takes, befriend...",1
3,"[john, boorman, zardoz, goofy, cinematic, debacle, fundamentally, misconceived, laughably, executed, takes, bizarre, enjoyment, quality, since, rampant, bumblings, one, edward, wood, jr, movie, si...",0
4,"[pulp, fiction, quentin, tarantino, anxiously, awaited, superb, follow, reservoir, dogs, absolutely, without, doubt, progressing, one, talked, loved, hated, films, time, fairness, noted, love, fil...",1


## Reformat reviews for sklearn

In [42]:
documents = []
for review in df_nltk.text:
  review = " ".join(review)
  documents.append(review)
  
sentiment = list(df_nltk.sentiment)
new_df = pd.DataFrame({'text': documents, 'sentiment': sentiment})
new_df.head()

,sentiment,text
0,0,saw film christmas day expecting upbeat comedy boy christmas dissapointment hour movie ready change rooms another theater read see say four rooms starring tim roth jennifer beals antonio banderas ...
1,0,first foremost think movie comes grief hands pg rating given us read james lee burke novel heaven prisoners based rabidly go buy tickets movies featuring combined talents alec baldwin eric roberts...
2,1,synopsis retiring detective jerry black nicholson becomes involved case murdered girl promises parents find killer purchases gas station route believes killer takes befriends lori wright penn youn...
3,0,john boorman zardoz goofy cinematic debacle fundamentally misconceived laughably executed takes bizarre enjoyment quality since rampant bumblings one edward wood jr movie silly serious time course...
4,1,pulp fiction quentin tarantino anxiously awaited superb follow reservoir dogs absolutely without doubt progressing one talked loved hated films time fairness noted love film greatly outnumber hate...


## Train Test Split

In [43]:
X = new_df.text
y = new_df.sentiment

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## Vectorize the reviews

In [44]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(max_features=None, ngram_range=(1,1), stop_words='english')

vectorizer.fit(X_train)

print(vectorizer.vocabulary_)


{'comedy': 5765, 'rated': 25081, 'pg': 23007, 'runs': 26829, 'starring': 29820, 'john': 16665, 'goodman': 13069, 'kathy': 16928, 'moriarty': 20277, 'bunch': 3968, 'teenagers': 31309, 'directed': 8374, 'joe': 16656, 'dante': 7363, 'written': 35089, 'charles': 4870, 'hass': 13937, 'summary': 30554, 'lawrence': 17649, 'woolsley': 34959, 'brings': 3717, 'new': 20960, 'horror': 14709, 'film': 11408, 'mant': 18934, 'premiere': 23957, 'key': 17031, 'west': 34548, 'height': 14145, 'cuban': 7084, 'missile': 19989, 'crisis': 6932, 'hopes': 14665, 'capitalize': 4365, 'tense': 31410, 'moment': 20141, 'providing': 24463, 'escape': 10322, 'town': 32115, 'events': 10455, 'stories': 30087, 'life': 17981, 'affects': 497, 'quick': 24790, 'easy': 9484, 'review': 26173, 'really': 25189, 'enjoyed': 10068, 'matinee': 19194, 'mixture': 20044, 'tension': 31411, 'blended': 3050, 'nicely': 21012, 'unlike': 33222, 'comedies': 5764, 'tries': 32395, 'succeeds': 30454, 'getting': 12717, 'past': 22526, 'stage': 2971

In [45]:
train_word_counts = vectorizer.transform(X_train)

X_train_vectorized = pd.DataFrame(train_word_counts.toarray(), columns=vectorizer.get_feature_names())

print(X_train_vectorized.shape)
X_train_vectorized.head()

(1600, 35427)


,aa,aaa,aaaaaaaaah,aaaaaaaahhhh,aaaaaah,aahs,aaliyah,aalyah,aamir,aardman,...,zuehlke,zuko,zukovsky,zulu,zundel,zus,zweibel,zwick,zwigoff,zzzzzzz
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [46]:
test_word_counts = vectorizer.transform(X_test)

X_test_vectorized = pd.DataFrame(test_word_counts.toarray(), columns=vectorizer.get_feature_names())

print(X_test_vectorized.shape)
X_test_vectorized.head()

(400, 35427)


,aa,aaa,aaaaaaaaah,aaaaaaaahhhh,aaaaaah,aahs,aaliyah,aalyah,aamir,aardman,...,zuehlke,zuko,zukovsky,zulu,zundel,zus,zweibel,zwick,zwigoff,zzzzzzz
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Logistic Regression

In [47]:
LR = LogisticRegression(random_state=42).fit(X_train_vectorized, y_train)

train_predictions = LR.predict(X_train_vectorized)
test_predictions = LR.predict(X_test_vectorized)

print(f'Train Accuracy: {accuracy_score(y_train, train_predictions)}')
print(f'Test Accuracy: {accuracy_score(y_test, test_predictions)}')

Train Accuracy: 0.9825
Test Accuracy: 0.815


## Multinomial Naive Bayes

In [48]:
MNB = MultinomialNB().fit(X_train_vectorized, y_train)

train_predictions = MNB.predict(X_train_vectorized)
test_predictions = MNB.predict(X_test_vectorized)

print(f'Train Accuracy: {accuracy_score(y_train, train_predictions)}')
print(f'Test Accuracy: {accuracy_score(y_test, test_predictions)}')

Train Accuracy: 0.970625
Test Accuracy: 0.815


## Random Forest Classifier

In [49]:
RFC = RandomForestClassifier().fit(X_train_vectorized, y_train)

train_predictions = RFC.predict(X_train_vectorized)
test_predictions = RFC.predict(X_test_vectorized)

print(f'Train Accuracy: {accuracy_score(y_train, train_predictions)}')
print(f'Test Accuracy: {accuracy_score(y_test, test_predictions)}')

Train Accuracy: 0.98875
Test Accuracy: 0.71


In [53]:
# import xgboost as xgb
from xgboost.sklearn import XGBClassifier

clf = XGBClassifier(
        #hyper params
        n_jobs = -1,
)

clf.fit(X_train, y_train, eval_metric = 'auc')

IndexError: tuple index out of range